# Лабораторная работа 6

**Задача:** В Швейцарии 4 диалекта немецкого языка. Цель лабораторной работы -- предсказать диалект по фрагменту разговорной речи.

**Данные:**

Описание файлов:
- train.csv - обучающая выборка
- test.csv - тестовая выборка
- sampleSubmission.csv - пример результирующего файла для его сабмита на кэгл

Описание колонок:
- Id - уникальный индекс для тестовой выборки (test.csv)
- Text - фрагмент речи
- Label - целевой класс языка для конкретного фрагмента речи (train.csv)
- Prediction - предсказанный класс языка (sampleSubmission.csv)

Описание классов:

`BE Bernese
 LU Lucerne
 ZH Zurich
 BS Basel`
    
    
**Подробнее о данных:** https://www.kaggle.com/competitions/swiss-dialect-identification/data 

In [18]:
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
import pandas as pd
from sklearn.metrics import classification_report
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np

1. Загрузить датафреймы через pandas. Выполнить предобработку `Bag of Words`.

In [66]:
test=pd.read_csv("data/test.csv",index_col=0)
train=pd.read_csv("data/train.csv")
sample_sub=pd.read_csv("data/sampleSubmission.csv",index_col=0)

In [20]:
vectorizer = CountVectorizer()
train_data = vectorizer.fit_transform(train["Text"])

In [21]:
train_data, validation_data, y_train, y_validation = train_test_split(train_data.toarray(), train["Label"].values, random_state=42, test_size=.2);

In [22]:
len(y_validation)/len(y_train)

0.250080051232789

2. Обучить различные модели `Random forest`, `Gradient boosting`, `SGD classifier` и тд (можно попробовать полносвязную нейронную сеть), снять метрику `accuracy` на валидационных данных.

## Random Forest

In [23]:
tree_rand = RandomForestClassifier()
tree_rand.fit(train_data,y_train)

RandomForestClassifier()

In [24]:
print("Accuracy: ",tree_rand.score(validation_data,y_validation))

Accuracy:  0.7103072983354674


## Gradient boosting

In [25]:
#gradient = GradientBoostingClassifier(validation_fraction=0.2,n_iter_no_change=5,n_estimators=20)
#gradient.fit(train_data,y_train)

In [26]:
#print("Accuracy: ",gradient.score(validation_data,y_validation))

## SGD Classifier

In [27]:
sgdc = SGDClassifier(max_iter=1000, tol=0.01)
sgdc.fit(train_data,y_train)

SGDClassifier(tol=0.01)

In [28]:
print("Accuracy: ",sgdc.score(validation_data,y_validation))

Accuracy:  0.7986555697823303


3. Выполнить предобработку `tf-idf`, поварировать параметры предобработки. Обучить модели. Как изменилось качество моделей с предобработкой `Bag of words` и `tf-idf`?

In [29]:
vectorizer = TfidfVectorizer()
train_data = vectorizer.fit_transform(train["Text"])

In [30]:
train_data, validation_data, y_train, y_validation = train_test_split(train_data.toarray(), train["Label"].values, random_state=42, test_size=.2);

In [31]:
len(y_validation)/len(y_train)

0.250080051232789

## Random Forest

In [32]:
tree_rand = RandomForestClassifier()
tree_rand.fit(train_data,y_train)

RandomForestClassifier()

In [33]:
print("Accuracy: ",tree_rand.score(validation_data,y_validation))

Accuracy:  0.721190781049936


## Gradient boosting

In [34]:
#gradient = GradientBoostingClassifier(validation_fraction=0.2,n_iter_no_change=5)
#gradient.fit(train_data,y_train)

In [35]:
#print("Accuracy: ",gradient.score(validation_data,y_validation))

## SGD Classifier

In [36]:
sgdc = SGDClassifier(max_iter=1000, tol=0.01)
sgdc.fit(train_data,y_train)

SGDClassifier(tol=0.01)

In [37]:
print("Accuracy: ",sgdc.score(validation_data,y_validation))

Accuracy:  0.7989756722151088


In [38]:
len(y_validation)/len(y_train)

0.250080051232789

4. Подумать как могут н-граммы слов или символов улучшить результата для предсказания языка. Попробовать составить токены n-gramm слов или символов и обучить выбранные модели на них. 

# Word

In [39]:
vectorizer = TfidfVectorizer(ngram_range=(1,2))
train_data = vectorizer.fit_transform(train["Text"])

In [40]:
len(train_data.toarray()[0])

81530

In [41]:
train_data, validation_data, y_train, y_validation = train_test_split(train_data.toarray(), train["Label"].values, random_state=42, test_size=.2);

#### SGD Classifier

In [42]:
sgdc = SGDClassifier(max_iter=1000, tol=0.01)
sgdc.fit(train_data,y_train)

SGDClassifier(tol=0.01)

In [43]:
print("Accuracy: ",sgdc.score(validation_data,y_validation))

Accuracy:  0.8021766965428937


# Char

In [44]:
vectorizer = TfidfVectorizer(ngram_range=(4,4), analyzer='char')
train_data = vectorizer.fit_transform(train["Text"])

In [45]:
train_data, validation_data, y_train, y_validation = train_test_split(train_data.toarray(), (train["Label"].values), random_state=42, test_size=.2);

#### SGD Classifier

In [46]:
sgdc = SGDClassifier(max_iter=1000, tol=0.01)
sgdc.fit(train_data,y_train)

SGDClassifier(tol=0.01)

In [47]:
print("Accuracy: ",sgdc.score(validation_data,y_validation))

Accuracy:  0.8229833546734955


5. Отправить лучший результат на https://www.kaggle.com/competitions/swiss-dialect-identification/overview

In [67]:
arr=vectorizer.transform(test[" Text"].to_list())

In [70]:
result=sgdc.predict(arr)

In [80]:
test[" Text"]=result

In [82]:
test.rename(columns = {' Text':'Prediction'}, inplace = True)

In [86]:
test.to_csv("submission.csv")